In [2]:
# import warnings
# warnings.filterwarnings("ignore")


import gc
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from imblearn.over_sampling import RandomOverSampler


import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow_hub as hub

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from pathlib import Path
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm as notebook_tqdm
from pkg_resources import parse_version



2025-10-25 09:46:48.657742: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 09:46:48.694938: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 09:46:49.721119: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


ModuleNotFoundError: No module named 'tensorflow_hub'

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
file_names = []
labels = []

data_path = Path("/home/uppercase/Workspace/Projects/Deepfake_Detection/dataset")
for file in sorted(data_path.glob("**/*.jpg")):
    label = file.parent.name
    labels.append(label)
    file_names.append(str(file))

print(f"Loaded {len(file_names)} images with {len(labels)} labels")

df = pd.DataFrame({"image": file_names, "label": labels})

Loaded 190335 images with 190335 labels


In [4]:
df.head()

,image,label
0,/home/uppercase/Workspace/Projects/Deepfake_De...,Fake
1,/home/uppercase/Workspace/Projects/Deepfake_De...,Fake
2,/home/uppercase/Workspace/Projects/Deepfake_De...,Fake
3,/home/uppercase/Workspace/Projects/Deepfake_De...,Fake
4,/home/uppercase/Workspace/Projects/Deepfake_De...,Fake


In [5]:
print(df['label'].unique())


['Fake' 'Real']


In [6]:
X = df.drop("label", axis=1)
y = df["label"]

ros = RandomOverSampler(random_state=83)
X_resampled, y_resampled = ros.fit_resample(X, y)

df = pd.DataFrame(X_resampled, columns=X.columns)
df["label"] = y_resampled

del X, y
gc.collect()

print(f"Resampled dataset shape: {df.shape}")

Resampled dataset shape: (190402, 2)


In [7]:
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(str)

In [ ]:
labels_list = ['Real', 'Fake']
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for i, label in enumerate(labels_list)}

df['label_int'] = df['label'].map(label2id)
df['label_str'] = df['label']

train_df = df.sample(frac=0.6, random_state=83)
test_df = df.drop(train_df.index)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.0
)

train_generator = datagen.flow_from_dataframe(
    train_df, x_col="image", y_col="label_str",
    target_size=(224, 224), batch_size=16, class_mode='categorical'
)

test_generator = datagen.flow_from_dataframe(
    test_df, x_col="image", y_col="label_str",
    target_size=(224, 224), batch_size=8, class_mode='categorical', shuffle=False
)

train_labels = train_df['label_int'].values
test_labels = test_df['label_int'].values

Found 114241 validated image filenames belonging to 2 classes.
Found 76161 validated image filenames belonging to 2 classes.


In [9]:

vit_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"
vit_module = hub.load(vit_url)

def vit_features(x):
    return vit_module(x)

def build_vit_model(num_classes=2):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = layers.Lambda(vit_features)(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

model = build_vit_model()
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


I0000 00:00:1761305402.860948    3575 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2111 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:02:00.0, compute capability: 8.6


In [10]:
gc.collect()

0

In [11]:
callbacks = [
    ModelCheckpoint('best_vit_model.h5', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
]

In [12]:
print("Train generator output shape:", next(train_generator)[1].shape)
print("Test generator output shape:", next(test_generator)[1].shape)

Train generator output shape: (16, 2)
Test generator output shape: (8, 2)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,  # Let generator handle labels
    epochs=12,
    callbacks=callbacks
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

/home/uppercase/Workspace/Projects/Deepfake_Detection/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12


2025-10-24 17:00:10.856229: I external/local_xla/xla/service/service.cc:163] XLA service 0x7d401400b500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-24 17:00:10.856263: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-10-24 17:00:11.028610: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-24 17:00:11.697145: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-10-24 17:00:13.269312: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_132', 8 bytes spill stores, 8 bytes spill loads

2025-10-24 17:00:13.928530: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346

   1/7141 ━━━━━━━━━━━━━━━━━━━━ 23:49:04 12s/step - accuracy: 0.4375 - loss: 1.6047

I0000 00:00:1761305419.209292    3830 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

7141/7141 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.5902 - loss: 1.3069

2025-10-24 17:15:55.737412: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_132', 8 bytes spill stores, 8 bytes spill loads

2025-10-24 17:15:55.954707: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_133', 528 bytes spill stores, 396 bytes spill loads

2025-10-24 17:15:56.765635: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7', 64 bytes spill stores, 64 bytes spill loads

2025-10-24 17:15:56.936581: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_109', 384 bytes spill stores, 384 bytes spill loads

2025-10-24 17:15:57.840233: I external/local

7141/7141 ━━━━━━━━━━━━━━━━━━━━ 1612s 224ms/step - accuracy: 0.6290 - loss: 1.1125 - val_accuracy: 0.7463 - val_loss: 0.5667 - learning_rate: 2.0000e-05
Epoch 2/12
7141/7141 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.7018 - loss: 0.7798

7141/7141 ━━━━━━━━━━━━━━━━━━━━ 1592s 223ms/step - accuracy: 0.7127 - loss: 0.7265 - val_accuracy: 0.7888 - val_loss: 0.4617 - learning_rate: 2.0000e-05
Epoch 3/12
7141/7141 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.7405 - loss: 0.6005

7141/7141 ━━━━━━━━━━━━━━━━━━━━ 1659s 232ms/step - accuracy: 0.7469 - loss: 0.5740 - val_accuracy: 0.8081 - val_loss: 0.4197 - learning_rate: 2.0000e-05
Epoch 4/12
7141/7141 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.7631 - loss: 0.5150

In [ ]:
# Define confusion matrix plotting function
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues, figsize=(10, 8)):
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    fmt = '.0f'
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
# Predictions
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = test_labels[:len(y_pred)]  # Use pre-extracted integer labels

# Metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Confusion Matrix
if len(labels_list) <= 150:
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, labels_list, figsize=(8, 6))